# Fully connected Auto-Encoder for Tabular Data
In order to compare the classical and quantum methods we will implement several methods including the deep learning method, fully connected auto-encoder. 

In [16]:
# TODO: Make this an automated file, not copying code from "SVM on Iris and arrhythmia Datasets.ipynb"
# Data processing
import pandas as pd
import numpy as np
from collections import Counter
# Visualization
import matplotlib.pyplot as plt

# Data Sets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer

#Auto-encoder imports
import torchvision
from torchvision import datasets
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#DataSets that we can test
liver = 'https://archive.ics.uci.edu/ml/machine-learning-databases/liver-disorders/bupa.data'
arrhythmia = 'https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data'
vowel = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/vowel/vowel-context.data'
iris = load_iris()
breast_cancer = load_breast_cancer()

name_datasets_csv = {'arrhythmia': arrhythmia}
name_datasets_sklearn = {'iris_data': iris, 'breast_cancer_data': breast_cancer}

# TODO: some error because of the location of the file
# from ipynb.fs.full.EDA import vowelData
# from ipynb.fs.full.EDA import irisData
# from ipynb.fs.full.EDA import breastcancerData


## Vowel Dataset + Building Auto-Encoder

In [17]:
# #for vowels dataset
# #target = 'feature12'
# #for iris and breast_cancer
# target = 'target'

# print("The dataset selected has shape: " + str(df_.shape) + " and for each class "+ str(Counter(df_[target])))

In [18]:
vowel_data = pd.read_csv(vowel, index_col=False, delim_whitespace=True, header=None)

In [19]:
normal_data = vowel_data.copy()
anomaly_data = vowel_data.copy()
#isolate vowel 5 and 6
normal_data.iloc[:,-1] = np.where((normal_data.iloc[:, -1] == 5)|(normal_data.iloc[:,-1] == 6), 0, 1)
#normal_data.iloc[:,-1] = np.where((normal_data.iloc[:, -1] == 5), 0, 1)
normal_data = normal_data[normal_data.iloc[:,-1] == 0].to_numpy()


#isolate anomaly class
anomaly_data.iloc[:,-1] = np.where((anomaly_data.iloc[:, -1] == 10),1,0)
anomaly_data = anomaly_data[anomaly_data.iloc[:,-1] == 1].to_numpy()

In [20]:
n_zeros = normal_data.shape[0]
train_size = int(n_zeros * 0.9)
test_size = n_zeros - train_size
#select only collumn 6
train_data,test_data = torch.utils.data.random_split(normal_data[:,:], [train_size, test_size], generator=torch.Generator().manual_seed(42))
test_data = np.concatenate([test_data, anomaly_data[:,:]])

In [21]:
# Settings the dtype the first time didn't work, don't ask me why, only god knows :)
train_data = np.array(train_data, dtype=np.float32)
test_data = np.array(test_data, dtype=np.float32)

In [22]:
train_data.shape, test_data.shape

((162, 14), (108, 14))

In [23]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [24]:
# Define the autoencoder class
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Linear(hidden_size//2, hidden_size//4)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size//4, hidden_size//2),
            nn.ReLU(),
            nn.Linear(hidden_size//2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Set up the autoencoder model
input_size = 13  # The size of the input data
hidden_size = 12  # The number of neurons in the hidden layer
autoencoder = AutoEncoder(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.005)

# Train the autoencoder
num_epochs = 50

for epoch in range(num_epochs):
    for batch in train_loader:

        X = batch[:, :-1] # drop the label

        # Forward pass
        output = autoencoder(X)
        loss = criterion(output, X)
        
        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
    # Print the loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/50], Loss: 5.2953
Epoch [2/50], Loss: 5.1585
Epoch [3/50], Loss: 5.0270
Epoch [4/50], Loss: 4.8880
Epoch [5/50], Loss: 4.7314
Epoch [6/50], Loss: 4.5497
Epoch [7/50], Loss: 4.3311
Epoch [8/50], Loss: 4.0527
Epoch [9/50], Loss: 3.6964
Epoch [10/50], Loss: 3.2428
Epoch [11/50], Loss: 2.6819
Epoch [12/50], Loss: 2.0544
Epoch [13/50], Loss: 1.5901
Epoch [14/50], Loss: 1.5392
Epoch [15/50], Loss: 1.3667
Epoch [16/50], Loss: 1.0725
Epoch [17/50], Loss: 0.9517
Epoch [18/50], Loss: 0.9079
Epoch [19/50], Loss: 0.8445
Epoch [20/50], Loss: 0.7769
Epoch [21/50], Loss: 0.7381
Epoch [22/50], Loss: 0.6986
Epoch [23/50], Loss: 0.6437
Epoch [24/50], Loss: 0.6048
Epoch [25/50], Loss: 0.5793
Epoch [26/50], Loss: 0.5581
Epoch [27/50], Loss: 0.5442
Epoch [28/50], Loss: 0.5318
Epoch [29/50], Loss: 0.5222
Epoch [30/50], Loss: 0.5163
Epoch [31/50], Loss: 0.5122
Epoch [32/50], Loss: 0.5069
Epoch [33/50], Loss: 0.4992
Epoch [34/50], Loss: 0.4914
Epoch [35/50], Loss: 0.4832
Epoch [36/50], Loss: 0.4731
E

In [25]:
for batch in test_loader:
    X, y = batch[:, :-1], batch[:, -1]
    output = autoencoder(X)
    diffs = np.sum((X - output).detach().numpy()**2, axis=1)
    print("This is the result for batch:",np.mean(diffs[y == 0]), np.mean(diffs[y == 1])) 

This is the result for batch: 2.9574237 4.1467695
This is the result for batch: nan 2.9643471


/Users/pallii/opt/miniconda3/envs/datascience/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/pallii/opt/miniconda3/envs/datascience/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


In [26]:
n_zeros = sum(vowel_data[:, -1] == 0)
train_size = int(n_zeros * 0.9)
test_size = n_zeros - train_size
train_data, test_data = torch.utils.data.random_split(vowel_data[vowel_data[:, -1] == 0], [train_size, test_size], generator=torch.Generator().manual_seed(42))
train_data = np.array(train_data, dtype=np.float32)
test_data = np.array(test_data, dtype=np.float32)
test_data = np.concatenate([test_data, vowel_data[vowel_data[:, -1] == 1]])



# Settings the dtype the first time didn't work, don't ask me why, only god knows :)
train_data = np.array(train_data, dtype=np.float32)
test_data = np.array(test_data, dtype=np.float32)



train_loader = torch.utils.data.DataLoader(train_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)
# TODO: Does it make sense to include both 0 and 1 target values at training?

# Define the autoencoder class
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Linear(hidden_size//2, hidden_size//4)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size//4, hidden_size//2),
            nn.ReLU(),
            nn.Linear(hidden_size//2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Set up the autoencoder model
input_size = 13  # The size of the input data
hidden_size = 12  # The number of neurons in the hidden layer
autoencoder = AutoEncoder(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.005)

# Train the autoencoder
num_epochs = 50

for epoch in range(num_epochs):
    for batch in train_loader:

        X = batch[:, :-1] # drop the label

        # Forward pass
        output = autoencoder(X)
        loss = criterion(output, X)
        
        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
    # Print the loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

for batch in test_loader:
    X, y = batch[:, :-1], batch[:, -1]
    output = autoencoder(X)
    diffs = np.sum((X - output).detach().numpy()**2, axis=1)
    print("This is the result for batch:",np.mean(diffs[y == 0]), np.mean(diffs[y == 1])) 


Epoch [1/50], Loss: 6.5719
Epoch [2/50], Loss: 5.3998
Epoch [3/50], Loss: 2.3187
Epoch [4/50], Loss: 0.7582
Epoch [5/50], Loss: 0.6399
Epoch [6/50], Loss: 0.5666
Epoch [7/50], Loss: 0.5499
Epoch [8/50], Loss: 0.5376
Epoch [9/50], Loss: 0.5285
Epoch [10/50], Loss: 0.5168
Epoch [11/50], Loss: 0.5023
Epoch [12/50], Loss: 0.4876
Epoch [13/50], Loss: 0.4732
Epoch [14/50], Loss: 0.4607
Epoch [15/50], Loss: 0.4496
Epoch [16/50], Loss: 0.4406
Epoch [17/50], Loss: 0.4333
Epoch [18/50], Loss: 0.4271
Epoch [19/50], Loss: 0.4224
Epoch [20/50], Loss: 0.4188
Epoch [21/50], Loss: 0.4159
Epoch [22/50], Loss: 0.4148
Epoch [23/50], Loss: 0.4125
Epoch [24/50], Loss: 0.4097
Epoch [25/50], Loss: 0.4077
Epoch [26/50], Loss: 0.4016
Epoch [27/50], Loss: 0.3922
Epoch [28/50], Loss: 0.3819
Epoch [29/50], Loss: 0.3709
Epoch [30/50], Loss: 0.3589
Epoch [31/50], Loss: 0.3471
Epoch [32/50], Loss: 0.3346
Epoch [33/50], Loss: 0.3249
Epoch [34/50], Loss: 0.3158
Epoch [35/50], Loss: 0.3088
Epoch [36/50], Loss: 0.3033
E

## Iris Dataset

In [32]:
i_normal_data = iris.copy()
i_anomaly_data = iris.copy()

In [33]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [ ]:
#